In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import os
from tqdm.notebook import tqdm
import ast


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import json
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
!python3.10 -m pip install chardet==3.0.4
!python3.10 -m pip install nltk==3.6.6
!python3.10 -m pip install numpy
!python3.10 -m pip install pandas
!python3.10 -m pip install xmltodict==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [7]:
#### Reading Data variables
labels = ["B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
          "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
          "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
          "I-STREET_ADDRESS","I-URL_PERSONAL","O"]
id2label = dict(enumerate(labels)) # integer label to BIO format label mapping
label2id = {v:k for k,v in id2label.items()} # BIO format label to integer label mapping
num_labels = len(labels) # number of PII (NER) tags
BASE_PATH = "./data/training_data/pii-detection-removal-from-educational-data"

In [8]:
# CREATING CUSTOM CONFIG
with open("./configs/philter_delta.json") as f:
    config = json.load(f)
    # titles below are from the philter config,in order, corresponding to the 'labels' given in the PII essays' data.
    relevant_titles = ["emails", "email header",
                      "id safe", "id verbose",
                       "six or more digits", "order number", "specimen number", "DDD-DD-DD", "DD-DDDDDC", "****DC", "model and serial numbers",
                       "confirmed by name", "editor name", "name_age", "names patterns", "name indicator", "Find Names 1", "Find Names 2", "Find Names 3", "Find Names 4",
                       "call #", 
                       "full street address", "full street address with concatenated street indicator", "number streetname city", "number streetname noindicator suite", "number streetname city state", "filters/regex/addresses/num_streetname_extension_transformed.txt", "number streetname san", "number streetname", "corner of street & street", "street and street", "at street number dash street", "at dash street", "short street name", "streetname floor number", "city state zip", "city zip", "lives in", "city state", "county name", "in city", "city", "box room", "room box", "floor box", "desk #", "at location", "to state", "state indicator", "on streetname", "box", "room",
                       "url", 
                       
                      ]
    relevant_config = []
    for con in config:
        if con["title"] in relevant_titles:
            relevant_config.append(con)
    with open("./configs/philter_delta_relevant.json", "w") as f_relevant:
        json.dump(relevant_config, f_relevant)

In [32]:
# Train-Valid data
data = json.load(open(f"{BASE_PATH}/train.json"))
pii_external_dataset =pd.read_csv(f"./data/training_data/pii-external-dataset/pii_dataset.csv")
data_mistral = json.load(open(f"./data/training_data/pii-dd-mistral-generated/mixtral-8x7b-v1.json"))
data_mpware = json.load(open("./data/training_data/pii-mixtral8x7b-generated-essays/mpware_mixtral8x7b_v1.1.json"))

# Initialize empty arrays
# words = np.empty(len(data) + len(pii_external_dataset) + len(data_mistral), dtype=object)
texts = np.empty(len(data) + len(pii_external_dataset) + len(data_mistral) + len(data_mpware), dtype=object)
# labels = np.empty(len(data) + len(pii_external_dataset) + len(data_mistral), dtype=object)

# Fill the arrays
for i, x in tqdm(enumerate(data), total=len(data)):
    texts[i] = (x["full_text"])
#     words[i] = np.array(x["tokens"])
#     labels[i] = np.array([label2id[label] for label in x["labels"]])

  0%|          | 0/6807 [00:00<?, ?it/s]

In [33]:
print(f"Length of pii_external_dataset : {len(pii_external_dataset)}")
# Fill the arrays
for ind in range(len(pii_external_dataset)):
    x = pii_external_dataset.iloc[ind]
    texts[ind + len(data)] = x["text"]
#     x_tokens_list = ast.literal_eval(x["tokens"])
#     x_labels_list = ast.literal_eval(x["labels"])
#     words[ind + len(data)] = np.array(x_tokens_list)
#     labels[ind + len(data)] = np.array([label2id[label] for label in x_labels_list])

Length of pii_external_dataset : 4434


In [34]:
print(f"Length of mistral_dataset : {len(data_mistral)}")
for i, x in tqdm(enumerate(data_mistral), total=len(data_mistral)):
#       print(np.array(x["tokens"]))
#       print(np.array([CFG.label2id[label] for label in x["labels"]]))
    texts[i + len(data) + len(pii_external_dataset)] = x["full_text"]
#     words[i + len(data) + len(pii_external_dataset)] = np.array(x["tokens"])
#     labels[i + len(data) + len(pii_external_dataset)] = np.array([label2id[label] for label in x["labels"]])

Length of mistral_dataset : 2355


  0%|          | 0/2355 [00:00<?, ?it/s]

In [38]:
for i, x in tqdm(enumerate(data_mpware), total=len(data_mpware)):
#       print(np.array(x["tokens"]))
#       print(np.array([CFG.label2id[label] for label in x["labels"]]))
    texts[i + len(data) + len(pii_external_dataset) + len(data_mistral)] = np.array(x["full_text"])
    # labels[i + len(data) + len(pii_external_dataset) + len(data_mistral)] = np.array([CFG.label2id[label] for label in x["labels"] if label != "I-USERNAME"])

  0%|          | 0/2692 [00:00<?, ?it/s]

In [39]:
os.getcwd()

'/Volumes/Akhil/GitHubRepos/philter-ucsf'

In [43]:
# shutil.rmtree("./data/data_essays/essays")

In [44]:
len(texts)

16288

In [48]:
# Data creation
if not os.path.exists("./data/data_essays"):
    os.mkdir("./data/data_essays")
if not os.path.exists("./data/data_essays/essays"):
    os.mkdir("./data/data_essays/essays")
if not os.path.exists("./data/data_essays/essays_results"):
    os.mkdir("./data/data_essays/essays_results")
for ind in range(len(texts)):
    if ind != 0 and ind % 5 == 0:
        print("Processing file : ", ind, "/", len(texts))
        break
    with open(f"./data/data_essays/essays/essay_{ind:08d}.txt", "w") as f:
        f.write(texts[ind])

Processing file :  5 / 16288


In [49]:
os.listdir("./data/data_essays/essays")

['essay_00000004.txt',
 'essay_00000002.txt',
 'essay_00000003.txt',
 'essay_00000001.txt',
 'essay_00000000.txt']

In [53]:
open("./data/data_essays/essays/essay_00000000.txt").read()

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages exchanges\n\n•  It makes visi

In [57]:
os.listdir("./data/data_essays/essays_results/")

['essay_00000004.txt',
 'essay_00000002.txt',
 'essay_00000003.txt',
 'essay_00000001.txt',
 'essay_00000000.txt',
 'essay_00000004.xml',
 'essay_00000003.xml',
 'essay_00000002.xml',
 'essay_00000000.xml',
 'essay_00000001.xml']

In [58]:
import xml.dom.minidom

In [59]:
# xml_fname = './data/data_essays/essays_results/essay_00000000.xml'
# dom = xml.dom.minidom.parse(xml_fname) # or xml.dom.minidom.parseString(xml_string)
# pretty_xml_as_string = dom.toprettyxml()
# print(pretty_xml_as_string)

with open('./data/data_essays/essays_results/essay_00000000.txt') as f:
    print(f.read())

Design Thinking for innovation reflexion-***** 2021-******** *****

Challenge & selection

The tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.

What exactly is a mind map? According to the definition of ***** *. and ***** *. (1999, *******-***  l'intelligence. *****: Les É******* d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf ******

This tool has many advantages:

•  It is accessible to all and does not require significant material investment and can be done  quickly

•  It is scalable

•  It allows categorization and linking of information

•  It can be applied to any type of situation: **********, problem solving, analysis, creation of  new ideas

•  It is suitable for all people and is easy to learn

•  It is fun and encourages exchanges

•  It makes visible the dimension of pr